In [5]:
!pip install  --quiet google-genai

In [9]:
!pip install gTTS --quiet

In [7]:
!pip install playsound

  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=cc453a3440f14eef79701fbfdc53e0175d88bd9473db75e130f694d78b3ad3c3
  Stored in directory: /root/.cache/pip/wheels/90/89/ed/2d643f4226fc8c7c9156fc28abd8051e2d2c0de37ae51ac45c
Successfully built playsound


In [12]:
from google.colab import userdata
GEMINI_API_KEY :str= userdata.get('video')
if(GEMINI_API_KEY):
  print("API KEY FOUND")
else:
  print("API KEY NOT FOUND")

API KEY FOUND


In [13]:
from google import genai
from google.genai import Client

client: Client = genai.Client(
    api_key=GEMINI_API_KEY,
)

model:str="gemini-2.0-flash-exp"

In [27]:
video_file_name="/content/sample_video.mp4"

In [28]:
import time

def upload_video(video_file_name):
  video_file=client.files.upload(path=video_file_name)
  while video_file.state=="PROCESSING":
    print("Wiating for video to processed")
    time.sleep(10)
    video_file=client.files.get(name=video_file.name or "")

  if video_file.state=="FAILED":
    raise ValueError(video_file.state)
  print(f"Video proceesing complete :" + (video_file.uri or ""))

  return video_file

my_video=upload_video(video_file_name)

Wiating for video to processed
Video proceesing complete :https://generativelanguage.googleapis.com/v1beta/files/vk78s9es9c3a


In [31]:
from google.genai.types import Content,Part
Prompt="""For each scene or action in  this video ,captions
          that describe the scene along with any text spoken text poalced in quotation marks. Place
          each caption into an object with the time code of the caption in the video
          """

video=my_video

response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""
                ),
            ]
        ),
        Prompt,
    ]

)
print(response.text)

```json
[
  {
    "time": "0:00",
    "caption": "A cartoon rocket stands against a blue and purple gradient background. Its top has a black rim with a blue antenna on top of a dome. The rocket body is light blue and has a rectangular black port near the middle with a downward arrow below."
  },
    {
    "time": "0:01",
   "caption": "The cartoon rocket is shown fully, it's blue bottom stands on a light blue patch of ground."
  },
  {
    "time": "0:02",
    "caption": "A text box appears that says, \"This is my #GitHubUnwrapped im-muhammadumair\" with a picture of a man in the upper left hand corner."
  },
  {
   "time": "0:03",
  "caption": "A text box appears that says \"My Top Language\" with a logo of a light blue animal with the letters \"GO\" on it."
  },
  {
   "time": "0:04",
  "caption": "A text box appears that says \"My Top Language\" with a logo of a dark gray rock."
 },
    {
   "time": "0:05",
    "caption": "A text box appears that says \"My Top Language\" with a logo 

In [37]:
from gtts import gTTS
from playsound import playsound

from IPython.display import Audio
tts=gTTS(text="\"Hello my name is Ghayyur Abbas and I am student of PIAIC Lahore and I am learning agentic AI , tHANKYOU",lang='en')

with open("output.mp3","wb") as f:
  for chunk in tts.stream():
    f.write(chunk)

display(Audio("output.mp3",autoplay=True))
